# filter()
- `DataFrameGroupBy.filter(func, dropna=True, *args, **kwargs)`
- 특정 조건을 만족하는 Group의 데이터(행)들을 조회할 때 사용한다. 주로 조건은 group별 집계결과를 이용한다.
    1. 함수에 group별 DataFrame을 argument로 전달한다.
    2. 함수는 받은 DataFrame을 이용해 집계한 값의 조건을 비교해서 반환한다.(반환타입: Bool) 
    3. 반환값이 True인 Group들의 모든 행들로 구성된 DataFrame을 반환한다.
- 매개변수
    - **func**: filtering 조건을 구현한 함수 객체
        - 첫번째 매개변수로 Group으로 묶인 DataFrame을 받는다.
        - bool type 값을 반환한다. 매개변수로 받은 DataFrame이 특정 조건을 만족하는지 여부를 반환한다.
    - **dropna=True**
        - 필터를 통과하지 못한 group의 DataFrame의 값들을 drop시킨다(기본값). False로 설정하면 NA 처리해서 반환한다.
    - **\*args, \*\*kwargs**: filter 함수의 두번째부터  선언된 매개변수에 전달할 argument 값들을 가변인자로 전달한다.

In [17]:
import numpy as np
import pandas as pd

# cnt1 값의 범위: 사과: 10대, 귤: 20대, 배: 단단위, 딸기 30이상
data = dict(fruits=['사과', '사과','사과', '사과','사과','귤','귤','귤','귤','귤','배','배','배','배','배','딸기','딸기','딸기','딸기','딸기']
            ,cnt1=[10, 12, 13, 11, 12, 21, 22, 27, 24, 26, 7, 7, 8, 3, 2, 30, 35, 37, 41, 28]
            ,cnt2=[100,  103, 107, 107,  101,  51,  57, 58,  57, 51,  9, 9,  5,  7,  7,  208, 217, 213, 206, 204]
           )
df = pd.DataFrame(data)
df

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


In [4]:
# cnt1의 값이 과일별 평균 이상인 값을 가지는 행들을 조회.
## 자기가 속한 group의 평균보다 큰값을 가지는 행들을 조회.

In [18]:
df.groupby("fruits").filter #(함수:조건을 체크하는 함수)

<bound method DataFrameGroupBy.filter of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x00000263F4A22430>>

In [19]:
# 조건을 체크하는 함수
## 파라미터 한개 이상을 정의 -> 첫번째 파라미터: Group별 DataFrame을 받을 변수.

def check_mean(x):
    # x 타입: DataFrame
    #  집계에대한 조건처리
    #데이터프레임을 받아서 처리한다고 보면 된다.
    return x['cnt1'].mean() > 20    # return 값: bool 

In [20]:
# cnt1의 평균이 20 이상인(check_mean) 과일들의 행을 조회 ==> filter()
df.groupby("fruits").filter(check_mean)
#어떤 식으로 쓰는지 그 메커니즘만 잘 알면 된다.

,fruits,cnt1,cnt2
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [21]:
df.groupby('fruits')['cnt1'].mean() > 20

fruits
귤      True
딸기     True
배     False
사과    False
Name: cnt1, dtype: bool

In [22]:
# filter함수
# 1번 파라미터(필수): DataFrame(group 별), 2번째 부터는 필요한대로 선언.
#  반환값: bool  True가 리턴되는 DataFrame들이 결과값.
def check_mean2(x, col_name, threshold):
    # X['col_name'].mean() >= threshold
    return x[col_name].mean() >= threshold

In [23]:
df.groupby('fruits').filter(check_mean2, col_name="cnt2", threshold=30)

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101
5,귤,21,51
6,귤,22,57
7,귤,27,58
8,귤,24,57
9,귤,26,51


In [24]:
df.groupby('fruits').filter(lambda x: x['cnt1'].mean() >= 30)

,fruits,cnt1,cnt2
15,딸기,30,208
16,딸기,35,217
17,딸기,37,213
18,딸기,41,206
19,딸기,28,204


In [25]:
df.groupby('fruits').filter(lambda x: x['cnt1'].mean() >= 20, dropna=False)

,fruits,cnt1,cnt2
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
5,귤,21.0,51.0
6,귤,22.0,57.0
7,귤,27.0,58.0
8,귤,24.0,57.0
9,귤,26.0,51.0


# transform
- `DataFrameGroupBy.transform(func, *args)`, `SeriesGroupBy.transform(func, *args)`
    - 함수(func)에 열의 값들을 group 별로 전달 한다. 함수는 그 값을 받아 통계량을 구해 반환한다. 반환된 통계량으로 원래 값들을 변경한 Series를 반환한다. 여러 컬럼에 대해 처리할 경우 DataFrame을 반환한다.
    - func: 함수객체
        - 매개변수
            - 그룹별 컬럼값들을 받을 변수 선언
        - return
            - 계산한 통계량.
        - DataFrameGroupBy은 모든 컬럼의 값들을 group 별 Series로 전달한다.
    - *args: 함수에 전달할 추가 인자값이 있으면 매개변수 순서에 맞게 값을 전달한다.
- transform() 함수를 groupby() 와 사용하면 컬럼의 각 원소들을 자신이 속한 그룹의 통계량으로 변환된 데이터셋을 생성할 수 있다.
- 컬럼의 값과 통계값을 비교해서 보거나 결측치 처리등에 사용할 수있다.

In [27]:
df.groupby('fruits')['cnt1'].transform("mean") # 판다스 제공 집계함수: 문자열로 전달.
#출력되는 형식에 대해 잘 알아두도록 하자.

0     11.6
1     11.6
2     11.6
3     11.6
4     11.6
5     24.0
6     24.0
7     24.0
8     24.0
9     24.0
10     5.4
11     5.4
12     5.4
13     5.4
14     5.4
15    34.2
16    34.2
17    34.2
18    34.2
19    34.2
Name: cnt1, dtype: float64

## 원본에 통계치 붙여서 비교하기

In [28]:
df2 = df.copy()
df2.head()

,fruits,cnt1,cnt2
0,사과,10,100
1,사과,12,103
2,사과,13,107
3,사과,11,107
4,사과,12,101


In [30]:
result = df2.groupby('fruits')['cnt1'].transform("mean") #'평균을 구한 것으로 데이터들을 채워라.'라는 의미이다.
result

0     11.6
1     11.6
2     11.6
3     11.6
4     11.6
5     24.0
6     24.0
7     24.0
8     24.0
9     24.0
10     5.4
11     5.4
12     5.4
13     5.4
14     5.4
15    34.2
16    34.2
17    34.2
18    34.2
19    34.2
Name: cnt1, dtype: float64

In [35]:
df2.insert(2, "cnt1 과일별 평균", result) #insert을 통해서 데이터를 새롭게 추가한다.
#주의사항:추가한 곳에 또 추가하진 말고ㅋㅋ

ValueError: cannot insert cnt1 과일별 평균, already exists

In [34]:
df2

,fruits,cnt1,cnt1 과일별 평균,cnt2
0,사과,10,11.6,100
1,사과,12,11.6,103
2,사과,13,11.6,107
3,사과,11,11.6,107
4,사과,12,11.6,101
5,귤,21,24.0,51
6,귤,22,24.0,57
7,귤,27,24.0,58
8,귤,24,24.0,57
9,귤,26,24.0,51


In [37]:
df2['cnt2 과일별 평균'] = df2.groupby("fruits")['cnt2'].transform('mean') #데이터의 형식을 잘 보도록 하자.
df2

,fruits,cnt1,cnt1 과일별 평균,cnt2,cnt2 과일별 평균
0,사과,10,11.6,100,103.6
1,사과,12,11.6,103,103.6
2,사과,13,11.6,107,103.6
3,사과,11,11.6,107,103.6
4,사과,12,11.6,101,103.6
5,귤,21,24.0,51,54.8
6,귤,22,24.0,57,54.8
7,귤,27,24.0,58,54.8
8,귤,24,24.0,57,54.8
9,귤,26,24.0,51,54.8


In [44]:
#  행 섞기
#  df.sample() :DataFrame의 데이터를 sampling(표본추출) -> 일부 데이터만 추출
#     - 전체 데이터를 섞은 다음에 지정한 비율/개수 만큼 추출
#     - frac = 추출할 비율( 0 ~ 1 실수) / 개수 (정수)
df3 = df.sample(frac=1.0).reset_index(drop=True)
df3

,fruits,cnt1,cnt2
0,딸기,30,208
1,사과,13,107
2,귤,24,57
3,딸기,28,204
4,사과,11,107
5,딸기,37,213
6,배,2,7
7,귤,22,57
8,배,3,7
9,귤,26,51


In [45]:
df3['cnt2 mean'] = df3.groupby('fruits')['cnt2'].transform("mean")
df3

,fruits,cnt1,cnt2,cnt2 mean
0,딸기,30,208,209.6
1,사과,13,107,103.6
2,귤,24,57,54.8
3,딸기,28,204,209.6
4,사과,11,107,103.6
5,딸기,37,213,209.6
6,배,2,7,7.4
7,귤,22,57,54.8
8,배,3,7,7.4
9,귤,26,51,54.8


In [46]:
#  최대 최소 값의 차이
def min_max_diff(x):
    # tranform에 전달할 함수 -> 파라미터: Series
    # 반환값: 처리한 값.
    return x.max() - x.min()


In [48]:
df3.groupby('fruits')['cnt1'].agg(min_max_diff)

fruits
귤      6
딸기    13
배      6
사과     3
Name: cnt1, dtype: int64

In [50]:
df3['min-max'] = df3.groupby('fruits')['cnt1'].transform(min_max_diff)
df3

,fruits,cnt1,cnt2,cnt2 mean,min-max
0,딸기,30,208,209.6,13
1,사과,13,107,103.6,3
2,귤,24,57,54.8,6
3,딸기,28,204,209.6,13
4,사과,11,107,103.6,3
5,딸기,37,213,209.6,13
6,배,2,7,7.4,6
7,귤,22,57,54.8,6
8,배,3,7,7.4,6
9,귤,26,51,54.8,6


In [51]:
df3['min-max2'] = df3.groupby('fruits')['cnt1'].transform(lambda x : x.max() - x.min())
df3

,fruits,cnt1,cnt2,cnt2 mean,min-max,min-max2
0,딸기,30,208,209.6,13,13
1,사과,13,107,103.6,3,3
2,귤,24,57,54.8,6,6
3,딸기,28,204,209.6,13,13
4,사과,11,107,103.6,3,3
5,딸기,37,213,209.6,13,13
6,배,2,7,7.4,6,6
7,귤,22,57,54.8,6,6
8,배,3,7,7.4,6,6
9,귤,26,51,54.8,6,6


## 결측치 처리
- transform이용해서 결측치를 같은 과일별 평균값으로 변환
    - 전체 평균보다 좀더 정확할 수 있다.

In [52]:
import pandas as pd
import numpy as np

s = pd.Series([10, 20, np.nan, 30, np.nan])
s

0    10.0
1    20.0
2     NaN
3    30.0
4     NaN
dtype: float64

In [53]:
# 결측치를 제거 - dropna()
# 결측치를 다른값으로 대체 - fillna(대체할 값)
s.fillna(1000)  #모든 결측치를 1000 으로 대체

0      10.0
1      20.0
2    1000.0
3      30.0
4    1000.0
dtype: float64

In [54]:
replace_value = pd.Series([10000, 20000, 30000, 40000, 50000])
replace_value

0    10000
1    20000
2    30000
3    40000
4    50000
dtype: int64

In [56]:
s.fillna(replace_value)  # s의 결측치를 같은 index의 replace_value값으로 변경. 결측치가 아닌 값들은 변경안됨.

0       10.0
1       20.0
2    30000.0
3       30.0
4    50000.0
dtype: float64

In [58]:
# fillna() 상수: 결측치를 상수로 변경.
#          dictionary, series, dataframe(대상이 dataframe) - index별로 다른 값으로 대체.
s.fillna({2:10000000, 4:505050})

0          10.0
1          20.0
2    10000000.0
3          30.0
4      505050.0
dtype: float64

In [62]:
df4 = df.copy()
df4.loc[[0, 1, 5, 6, 10, 11, 15, 16], 'cnt2'] = np.nan
df4

,fruits,cnt1,cnt2
0,사과,10,NaN
1,사과,12,NaN
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
5,귤,21,NaN
6,귤,22,NaN
7,귤,27,58.0
8,귤,24,57.0
9,귤,26,51.0


In [67]:
# 결측치 처리
### 제거: Datafram: 행/열 단위로 제거. default: 행
df4.dropna()# axis=1)

,fruits,cnt1,cnt2
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
7,귤,27,58.0
8,귤,24,57.0
9,귤,26,51.0
12,배,8,5.0
13,배,3,7.0
14,배,2,7.0
17,딸기,37,213.0


In [68]:
### 다른 값으로 대체
df4.fillna(1000) # 컬럼과 상관없이 모든 결측치를 동일한 값으로 변경

,fruits,cnt1,cnt2
0,사과,10,1000.0
1,사과,12,1000.0
2,사과,13,107.0
3,사과,11,107.0
4,사과,12,101.0
5,귤,21,1000.0
6,귤,22,1000.0
7,귤,27,58.0
8,귤,24,57.0
9,귤,26,51.0


In [75]:
# round(df4['cnt2'].median())

80

In [77]:
# 결측치를 대체(가장 가능성 높은 값으로 변경. => 평균/중앙값, 최빈값)
df4['cnt2_1'] = df4['cnt2'].fillna(round(df4['cnt2'].mean()))
df4
# 전체 cnt2를 기준으로 평균을 계산해서 결측치를 대체. 
## 특정 카테고리별로(과일) 평균값의 차이가 클경우 전체 평균은 유용한 결측치 대체값이 아니다.
## 카테고리별로 평균을 구해 그 카테고리의 결측치 대체 값으로 사용한다.

,fruits,cnt1,cnt2,cnt2_1
0,사과,10,NaN,94.0
1,사과,12,NaN,94.0
2,사과,13,107.0,107.0
3,사과,11,107.0,107.0
4,사과,12,101.0,101.0
5,귤,21,NaN,94.0
6,귤,22,NaN,94.0
7,귤,27,58.0,58.0
8,귤,24,57.0,57.0
9,귤,26,51.0,51.0


In [82]:
df4.groupby('fruits')['cnt2'].mean()

fruits
귤      55.333333
딸기    207.666667
배       6.333333
사과    105.000000
Name: cnt2, dtype: float64

In [85]:
m = df4.groupby('fruits')['cnt2'].transform("mean")
df4['cnt2'] = df4['cnt2'].fillna(m)

In [86]:
df4

,fruits,cnt1,cnt2,cnt2_1
0,사과,10,105.000000,94.0
1,사과,12,105.000000,94.0
2,사과,13,107.000000,107.0
3,사과,11,107.000000,107.0
4,사과,12,101.000000,101.0
5,귤,21,55.333333,94.0
6,귤,22,55.333333,94.0
7,귤,27,58.000000,58.0
8,귤,24,57.000000,57.0
9,귤,26,51.000000,51.0


<b style='font-size:2em'>TODO </b>

In [87]:
# 1.  data/diamonds.csv 조회
dia = pd.read_csv('data/diamonds.csv')
dia.shape

(53940, 10)

In [88]:
# 2.  cut 별 평균 가격이 4000 이상인 diamond 데이터들 조회 
dia.groupby('cut')['price'].mean()

cut
Fair         4358.757764
Good         3928.864452
Ideal        3457.541970
Premium      4584.257704
Very Good    3981.759891
Name: price, dtype: float64

In [90]:
# x <- cut 별 DataFrame이 전달.
result = dia.groupby('cut').filter(lambda x : x['price'].mean() >= 4000)
result['cut'].unique()

array(['Premium', 'Fair'], dtype=object)

In [94]:
# 3. color 별 carat의 최대값과 최소값의 차이가 2이상 3미만인 모든 diamond 데이터들 조회
def min_max_check(dataframe):
    min_v = dataframe['carat'].min()
    max_v = dataframe['carat'].max()
    diff = max_v - min_v
    return diff >= 2 and diff < 3

result = dia.groupby('color').filter(min_max_check)

In [96]:
result.color.unique()

array(['E', 'F', 'G'], dtype=object)

In [100]:
dia.groupby('color')['carat'].agg(['min', 'max', lambda x: x.max()-x.min()])

,min,max,<lambda_0>
color,,,
D,0.20,3.40,3.20
E,0.20,3.05,2.85
F,0.20,3.01,2.81
G,0.23,3.01,2.78
H,0.23,4.13,3.90
I,0.23,4.01,3.78
J,0.23,5.01,4.78


In [103]:
dia.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y',
       'z'],
      dtype='object')

In [104]:
# 4. clarity 별 평균 가격 컬럼을 DataFrame에 추가.
r = dia.groupby('clarity')['price'].transform('mean')

In [105]:
dia.insert(7, 'price_mean', r)

In [106]:
dia.head(20)

,carat,cut,color,clarity,depth,table,price,price_mean,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,5063.028606,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3996.001148,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,3839.455391,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,3924.989395,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,5063.028606,4.34,4.35,2.75
5,0.24,Very Good,J,VVS2,62.8,57.0,336,3283.737071,3.94,3.96,2.48
6,0.24,Very Good,I,VVS1,62.3,57.0,336,2523.114637,3.95,3.98,2.47
7,0.26,Very Good,H,SI1,61.9,55.0,337,3996.001148,4.07,4.11,2.53
8,0.22,Fair,E,VS2,65.1,61.0,337,3924.989395,3.87,3.78,2.49
9,0.23,Very Good,H,VS1,59.4,61.0,338,3839.455391,4.00,4.05,2.39


# pivot_table()
엑셀의 pivot table 기능을 제공하는 메소드.    
분류별 집계(Group으로 묶어 집계)를 처리하는 함수로 group으로 묶고자 하는 컬럼들을 행과 열로 위치시키고 집계값을 값으로 보여준다.    
역할은 `groupby()`를 이용한 집계와 같은데 **여러개 컬럼을 기준으로 groupby 를 할 경우 집계결과를 읽는 것이 더 편하다.(가독성이 좋다)**

> pivot() 함수와 역할이 다르다.   
> pivot() 은 index와 column의 형태를 바꾸는 reshape 함수.

- `DataFrame.pivot_table(values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All')`
- **매개변수**
    - **index**
        - 문자열 또는 리스트. index로 올 컬럼들 => groupby였으면 묶었을 컬럼
    - **columns**
        - 문자열 또는 리스트. column으로 올 컬럼들 => groupby였으면 묶었을 컬럼 (index/columns가 묶여서 groupby에 묶을 컬럼들이 된다.)
    - **values**
        - 문자열 또는 리스트. 집계할 대상 컬럼들
    - **aggfunc**
        - 집계함수 지정. 함수, 함수이름문자열, 함수리스트(함수이름 문자열/함수객체), dict: 집계할 함수
        - 기본(생략시): 평균을 구한다. (mean이 기본값)
    - **fill_value, dropna**
        - fill_value: 집계시 NA가 나올경우 채울 값
        - dropna: boolean. 컬럼의 전체값이 NA인 경우 그 컬럼 제거(기본: True)
    - **margins/margins_name**
        - margin: boolean(기본: False). 총집계결과를 만들지 여부.
        - margin_name: margin의 이름 문자열로 지정 (생략시 All)

In [40]:
flight = pd.read_csv('saved_data/flights.csv')
flight.shape

(58492, 14)

In [41]:
flight.groupby("AIRLINE")['AIR_TIME'].mean()

AIRLINE
AA    144.259404
AS    147.845052
B6    209.412963
DL    115.334187
EV     68.964016
F9    127.592337
HA    338.288288
MQ     61.318346
NK    135.736878
OO     76.010272
UA    155.650521
US    147.686755
VX    154.864097
WN    107.005897
Name: AIR_TIME, dtype: float64

In [42]:
14 * 12

168

In [43]:
# 항공사, 월별 xxxx 평균
result = flight.groupby(['AIRLINE', 'MONTH'])['AIR_TIME'].mean()
print(result.shape)
result

(149,)


AIRLINE  MONTH
AA       1        138.793103
         2        143.126560
         3        142.026163
         4        144.544267
         5        145.261538
                     ...    
WN       7        109.377500
         8        109.173248
         9        109.842239
         11       104.704244
         12       104.136304
Name: AIR_TIME, Length: 149, dtype: float64

## 1개의 컬럼을 grouping 해서 집계
- 항공사별 비행시간의 평균 
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
    - 집계대상컬럼
        - AIR_TIME
- 집계: mean

In [44]:
flight.groupby('AIRLINE')['AIR_TIME'].mean()# .to_frame()

AIRLINE
AA    144.259404
AS    147.845052
B6    209.412963
DL    115.334187
EV     68.964016
F9    127.592337
HA    338.288288
MQ     61.318346
NK    135.736878
OO     76.010272
UA    155.650521
US    147.686755
VX    154.864097
WN    107.005897
Name: AIR_TIME, dtype: float64

In [45]:
# flight.pivot_table(index='AIRLINE', values='AIR_TIME', aggfunc='mean')
flight.pivot_table(columns='AIRLINE', values='AIR_TIME', aggfunc='mean')

AIRLINE,AA,AS,B6,DL,EV,F9,HA,MQ,NK,OO,UA,US,VX,WN
AIR_TIME,144.259404,147.845052,209.412963,115.334187,68.964016,127.592337,338.288288,61.318346,135.736878,76.010272,155.650521,147.686755,154.864097,107.005897


## 두개의 컬럼을 grouping 해서 집계
- 항공사/출발공항코드 별 취소 총수 (1이 취소이므로 합계를 구한다.)
- 사용컬럼
    - grouping할 컬럼
        - AIRLINE: 항공사
        - ORG_AIR: 출발 공항코드
    - 집계대상컬럼
        - CANCELLED: 취소여부 - 1:취소, 0: 취소안됨
- 집계: sum

In [46]:
flight.groupby(['AIRLINE', 'ORG_AIR'])['CANCELLED'].sum()

AIRLINE  ORG_AIR
AA       ATL         3
         DEN         4
         DFW        86
         IAH         3
         LAS         3
                    ..
WN       LAS         7
         LAX        32
         MSP         1
         PHX         6
         SFO        25
Name: CANCELLED, Length: 114, dtype: int64

In [47]:
flight.pivot_table(index='AIRLINE', # 그룹을 나누는 기준 컬럼중 INDEX(행)에 놓을 컬럼.
                   columns='ORG_AIR',# 그룹을 나누는 기준 컬럼중 COLUMN(열)에 놓을 컬럼.
                   values='CANCELLED', # 집계대상 컬럼
                   aggfunc="sum", # 집계 함수
                   fill_value='-999', # NaN을 대신할 값
                   margins=True, # sql의 rollup 
                   margins_name="총계" # margin 컬럼/index의 이름지정
                  )

ORG_AIR,ATL,DEN,DFW,IAH,LAS,LAX,MSP,ORD,PHX,SFO,총계
AIRLINE,,,,,,,,,,,
AA,3.0,4.0,86.0,3.0,3.0,11.0,3.0,35.0,4.0,2.0,154
AS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
B6,-999,0.0,0.0,-999,0.0,0.0,-999,0.0,0.0,1.0,1
DL,28.0,1.0,0.0,0.0,1.0,1.0,4.0,0.0,1.0,2.0,38
EV,18.0,6.0,27.0,36.0,-999,-999,6.0,53.0,0.0,-999,146
F9,0.0,2.0,1.0,0.0,1.0,1.0,1.0,4.0,0.0,0.0,10
HA,-999,-999,-999,-999,0.0,0.0,-999,-999,0.0,0.0,0
MQ,5.0,-999,62.0,0.0,-999,0.0,0.0,85.0,-999,-999,152
NK,1.0,1.0,6.0,0.0,1.0,1.0,3.0,10.0,2.0,-999,25


In [118]:
# NaN : 그룹으로 묶인 데이터가 없은 경우.
flight.query('AIRLINE=="B6" and ORG_AIR=="IAH"')

,MONTH,DAY,WEEKDAY,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED


## 3개 이상의 컬럼을 grouping해서 집계
- 항공사/월/출발공항코드 별 취소 총수 
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - CANCELLED: 취소여부
- 집계 : sum    

In [126]:
flight.pivot_table(index="MONTH", 
                   columns=['AIRLINE', 'ORG_AIR'], # 묶을 대상이 여러개인 경우 리스트로.
                   values="CANCELLED",
                   aggfunc="sum"
                  )

AIRLINE   AA                                                ...   VX       \
ORG_AIR  ATL  DEN   DFW  IAH  LAS  LAX  MSP  ORD  PHX  SFO  ...  LAX  ORD   
MONTH                                                       ...             
1        0.0  0.0   8.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2        2.0  1.0  33.0  1.0  2.0  3.0  0.0  7.0  1.0  1.0  ...  2.0  0.0   
3        1.0  0.0  13.0  0.0  0.0  1.0  0.0  1.0  0.0  0.0  ...  0.0  NaN   
4        0.0  0.0   4.0  0.0  0.0  0.0  0.0  5.0  0.0  0.0  ...  1.0  0.0   
5        0.0  2.0   8.0  1.0  0.0  0.0  0.0  2.0  0.0  0.0  ...  0.0  0.0   
6        0.0  0.0   7.0  0.0  0.0  0.0  0.0  4.0  0.0  0.0  ...  0.0  0.0   
7        0.0  0.0   1.0  0.0  1.0  0.0  1.0  3.0  2.0  1.0  ...  0.0  0.0   
8        0.0  0.0   2.0  1.0  0.0  2.0  0.0  3.0  1.0  0.0  ...  0.0  0.0   
9        0.0  0.0   1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
11       0.0  0.0   3.0  0.0  0.0  1.0  1.0  6.0  0.0  0.0  ...  0.0  0.0   
12       0.0  1.0   6.0  0.0  0.0  2.0  1.0  4.0  0.0  0.0  ...  0.0  0.0   

AIRLINE        WN                                
ORG_AIR  SFO  ATL  DEN  LAS  LAX  MSP  PHX  SFO  
MONTH                                            
1        1.0  3.0  0.0  1.0  3.0  1.0  0.0  4.0  
2        1.0  3.0  3.0  1.0  2.0  0.0  2.0  5.0  
3        0.0  1.0  0.0  1.0  3.0  0.0  1.0  0.0  
4        0.0  0.0  4.0  0.0  2.0  0.0  1.0  2.0  
5        0.0  0.0  1.0  0.0  1.0  0.0  0.0  2.0  
6        0.0  1.0  1.0  0.0  0.0  0.0  0.0  6.0  
7        1.0  0.0  0.0  1.0  9.0  0.0  1.0  0.0  
8        0.0  0.0  0.0  1.0  4.0  0.0  0.0  2.0  
9        0.0  0.0  0.0  0.0  3.0  0.0  0.0  2.0  
11       0.0  1.0  3.0  1.0  3.0  0.0  1.0  2.0  
12       0.0  0.0  1.0  1.0  2.0  0.0  0.0  0.0  

[11 rows x 114 columns]

In [130]:
flight.pivot_table(columns="MONTH", 
                   index=['AIRLINE', 'ORG_AIR'], # 묶을 대상이 여러개인 경우 리스트로.
                   values="CANCELLED",
                   aggfunc="sum", 
                   margins=True
                  )

MONTH                1      2     3     4     5      6     7     8     9  \
AIRLINE ORG_AIR                                                            
AA      ATL        0.0    2.0   1.0   0.0   0.0    0.0   0.0   0.0   0.0   
        DEN        0.0    1.0   0.0   0.0   2.0    0.0   0.0   0.0   0.0   
        DFW        8.0   33.0  13.0   4.0   8.0    7.0   1.0   2.0   1.0   
        IAH        0.0    1.0   0.0   0.0   1.0    0.0   0.0   1.0   0.0   
        LAS        0.0    2.0   0.0   0.0   0.0    0.0   1.0   0.0   0.0   
...                ...    ...   ...   ...   ...    ...   ...   ...   ...   
WN      LAX        3.0    2.0   3.0   2.0   1.0    0.0   9.0   4.0   3.0   
        MSP        1.0    0.0   0.0   0.0   0.0    0.0   0.0   0.0   0.0   
        PHX        0.0    2.0   1.0   1.0   0.0    0.0   1.0   0.0   0.0   
        SFO        4.0    5.0   0.0   2.0   2.0    6.0   0.0   2.0   2.0   
All              104.0  195.0  94.0  58.0  81.0  100.0  56.0  44.0  17.0   

MONTH              11    12  All  
AIRLINE ORG_AIR                   
AA      ATL       0.0   0.0    3  
        DEN       0.0   1.0    4  
        DFW       3.0   6.0   86  
        IAH       0.0   0.0    3  
        LAS       0.0   0.0    3  
...               ...   ...  ...  
WN      LAX       3.0   2.0   32  
        MSP       0.0   0.0    1  
        PHX       1.0   0.0    6  
        SFO       2.0   0.0   25  
All              51.0  81.0  881  

[115 rows x 12 columns]

## 3개 이상의 컬럼을 grouping해서 집계 2
- 항공사/월/출발공항코드 별 최대/최소 연착시간
- grouping할 컬럼
    - AIRLINE:항공사
    - MONTH:월
    - ORG_AIR: 출발지 공항
- 집계 대상컬럼
    - ARR_DELAY: 연착시간
- 집계 : min, max    

In [48]:
result_df = flight.pivot_table(index=['AIRLINE', 'ORG_AIR'],
                               columns='MONTH', 
                               values='ARR_DELAY',
                               aggfunc=['min', 'max'], # 집계함수가 여러개인 경우 list로 묶어 제공.
                              )
result_df

min                                                        \
MONTH              1     2     3     4     5     6     7     8     9     11   
AIRLINE ORG_AIR                                                               
AA      ATL     -27.0 -26.0 -32.0 -30.0 -26.0 -23.0 -27.0 -32.0 -33.0 -33.0   
        DEN     -13.0 -17.0 -19.0 -28.0 -20.0 -13.0 -30.0 -27.0 -27.0 -27.0   
        DFW     -39.0 -29.0 -29.0 -37.0 -36.0 -33.0 -32.0 -32.0 -33.0 -45.0   
        IAH     -23.0 -27.0 -13.0 -27.0 -19.0 -13.0 -19.0 -30.0 -31.0 -14.0   
        LAS     -32.0 -25.0 -24.0 -19.0 -31.0 -27.0 -40.0 -28.0 -32.0 -31.0   
...               ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
WN      LAS     -25.0 -33.0 -28.0 -38.0 -32.0 -31.0 -24.0 -24.0 -33.0 -40.0   
        LAX     -25.0 -27.0 -25.0 -37.0 -26.0 -30.0 -25.0 -20.0 -30.0 -28.0   
        MSP     -38.0 -32.0 -30.0 -33.0 -24.0 -24.0 -28.0 -21.0 -23.0 -29.0   
        PHX     -38.0 -45.0 -24.0 -43.0 -27.0 -25.0 -29.0 -24.0 -25.0 -41.0   
        SFO     -19.0 -24.0 -29.0 -29.0 -22.0 -26.0 -30.0 -22.0 -33.0 -23.0   

                 ...    max                                                   \
MONTH            ...     2      3      4      5      6      7      8      9    
AIRLINE ORG_AIR  ...                                                           
AA      ATL      ...   16.0   25.0  115.0   25.0  159.0  319.0   84.0  196.0   
        DEN      ...   40.0   53.0  105.0  330.0   10.0   67.0  257.0  152.0   
        DFW      ...  311.0  234.0  275.0  285.0  602.0  203.0  268.0  241.0   
        IAH      ...   51.0   97.0  127.0  131.0  456.0  858.0   95.0   73.0   
        LAS      ...   20.0  111.0  626.0   54.0  206.0  157.0  157.0   36.0   
...              ...    ...    ...    ...    ...    ...    ...    ...    ...   
WN      LAS      ...  135.0  261.0  163.0  150.0  193.0  290.0  284.0  284.0   
        LAX      ...  190.0  104.0  165.0  244.0  244.0  248.0  221.0  157.0   
        MSP      ...   68.0   43.0   64.0   13.0   88.0   84.0   45.0   23.0   
        PHX      ...  159.0   78.0  130.0  203.0  168.0  171.0  161.0  131.0   
        SFO      ...  110.0  237.0   72.0  133.0  205.0   51.0   65.0   69.0   

                               
MONTH               11     12  
AIRLINE ORG_AIR                
AA      ATL      255.0  203.0  
        DEN      146.0  106.0  
        DFW      349.0  293.0  
        IAH       98.0  103.0  
        LAS       89.0  219.0  
...                ...    ...  
WN      LAS      120.0   96.0  
        LAX       87.0  493.0  
        MSP      114.0   90.0  
        PHX      244.0  254.0  
        SFO      143.0  215.0  

[114 rows x 22 columns]

min                                                        \
MONTH              1     2     3     4     5     6     7     8     9     11   
AIRLINE ORG_AIR                                                               
AA      ATL     -27.0 -26.0 -32.0 -30.0 -26.0 -23.0 -27.0 -32.0 -33.0 -33.0   
        DEN     -13.0 -17.0 -19.0 -28.0 -20.0 -13.0 -30.0 -27.0 -27.0 -27.0   
        DFW     -39.0 -29.0 -29.0 -37.0 -36.0 -33.0 -32.0 -32.0 -33.0 -45.0   
        IAH     -23.0 -27.0 -13.0 -27.0 -19.0 -13.0 -19.0 -30.0 -31.0 -14.0   
        LAS     -32.0 -25.0 -24.0 -19.0 -31.0 -27.0 -40.0 -28.0 -32.0 -31.0   
...               ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
WN      LAS     -25.0 -33.0 -28.0 -38.0 -32.0 -31.0 -24.0 -24.0 -33.0 -40.0   
        LAX     -25.0 -27.0 -25.0 -37.0 -26.0 -30.0 -25.0 -20.0 -30.0 -28.0   
        MSP     -38.0 -32.0 -30.0 -33.0 -24.0 -24.0 -28.0 -21.0 -23.0 -29.0   
        PHX     -38.0 -45.0 -24.0 -43.0 -27.0 -25.0 -29.0 -24.0 -25.0 -41.0   
        SFO     -19.0 -24.0 -29.0 -29.0 -22.0 -26.0 -30.0 -22.0 -33.0 -23.0   

                 ...    max                                                   \
MONTH            ...     2      3      4      5      6      7      8      9    
AIRLINE ORG_AIR  ...                                                           
AA      ATL      ...   16.0   25.0  115.0   25.0  159.0  319.0   84.0  196.0   
        DEN      ...   40.0   53.0  105.0  330.0   10.0   67.0  257.0  152.0   
        DFW      ...  311.0  234.0  275.0  285.0  602.0  203.0  268.0  241.0   
        IAH      ...   51.0   97.0  127.0  131.0  456.0  858.0   95.0   73.0   
        LAS      ...   20.0  111.0  626.0   54.0  206.0  157.0  157.0   36.0   
...              ...    ...    ...    ...    ...    ...    ...    ...    ...   
WN      LAS      ...  135.0  261.0  163.0  150.0  193.0  290.0  284.0  284.0   
        LAX      ...  190.0  104.0  165.0  244.0  244.0  248.0  221.0  157.0   
        MSP      ...   68.0   43.0   64.0   13.0   88.0   84.0   45.0   23.0   
        PHX      ...  159.0   78.0  130.0  203.0  168.0  171.0  161.0  131.0   
        SFO      ...  110.0  237.0   72.0  133.0  205.0   51.0   65.0   69.0   

                               
MONTH               11     12  
AIRLINE ORG_AIR                
AA      ATL      255.0  203.0  
        DEN      146.0  106.0  
        DFW      349.0  293.0  
        IAH       98.0  103.0  
        LAS       89.0  219.0  
...                ...    ...  
WN      LAS      120.0   96.0  
        LAX       87.0  493.0  
        MSP      114.0   90.0  
        PHX      244.0  254.0  
        SFO      143.0  215.0  

[114 rows x 22 columns]

# apply() - Series, DataFrame의 데이터 일괄 처리

데이터프레임의 행들과 열들 또는 Series의 원소들에 공통된 처리를 할 때 apply 함수를 이용하면 반복문을 사용하지 않고 일괄 처리가 가능하다.

- DataFrame.apply(함수, axis=0, args=(), \*\*kwarg)
    - 인수로 행이나 열을 받는 함수를 apply 메서드의 인수로 넣으면 데이터프레임의 행이나 열들을 하나씩 함수에 전달한다.
    - 매개변수
        - **함수**: DataFrame의 행들 또는 열들을 전달할 함수
        - **axis**: **0-컬럼(열)을 전달, 1-행을 전달 (기본값 0)**
        - **args**: 함수에 행/열 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달
        - **\*\*kwarg**: 함수에 행/열 이외에 전달할 매개변수를 키워드 인자로 전달
- Series.apply(함수, args=(), \*\*kwarg)
    - 인수로 Series의 원소들을 받는 함수를 apply 메소드의 인수로 넣으면  Series의 원소들을 하나씩 함수로 전달한다.
    - 매개변수
        - **함수**: Series의 원소들을 전달할 함수
        - **args**: 함수에 원소 이외에 전달할 매개변수를 위치기반(순서대로) 튜플로 전달
        - **\*\*kwarg**: 함수에 원소 이외에 전달할 매개변수를 키워드 인자로 전달

In [140]:
# 1: 월  ~ 7:일
flight.WEEKDAY.value_counts().sort_index()

WEEKDAY
1    8720
2    8535
3    8706
4    8659
5    8453
6    7052
7    8367
Name: count, dtype: int64

In [141]:
# 요일 전체(일괄처리): 정수 -> 문자열 
flight.WEEKDAY

0        4
1        4
2        4
3        4
4        4
        ..
58487    4
58488    4
58489    4
58490    4
58491    4
Name: WEEKDAY, Length: 58492, dtype: int64

In [152]:
list("월화수목금토일")[2-1]

'화'

In [153]:
# Series의 원소 하나를 받아서 처리후 반환하는 함수
def change_weekday(value):
    l = list("월화수목금토일")
    v = l[value-1]
    return v+"요일"

In [148]:
change_weekday(7)

'일요일'

In [155]:
flight['WEEKDAY'] = flight['WEEKDAY'].apply(change_weekday)

In [162]:
# 요일: value
d = {
    v+"요일" : i for i, v in enumerate(list("월화수목금토일"), start=1)
}
d

{'월요일': 1, '화요일': 2, '수요일': 3, '목요일': 4, '금요일': 5, '토요일': 6, '일요일': 7}

In [163]:
d['월요일']

1

In [168]:
flight.columns

Index(['MONTH', 'DAY', 'WEEKDAY', 'AIRLINE', 'ORG_AIR', 'DEST_AIR',
       'SCHED_DEP', 'DEP_DELAY', 'AIR_TIME', 'DIST', 'SCHED_ARR', 'ARR_DELAY',
       'DIVERTED', 'CANCELLED'],
      dtype='object')

In [169]:
#  WEEKDAY: 문자열 -> 정수 INDEX
flight.insert(3, "WEEKDAY2", 
              flight['WEEKDAY'].apply(lambda x : d[x]))  # X: 원소 한개 -> 요일문자열. 
flight.head()


,MONTH,DAY,WEEKDAY,WEEKDAY2,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,목요일,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,목요일,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,목요일,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,목요일,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,목요일,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [183]:
# Dataframe.apply(함수)  # 함수(컬럼 또는 행: Series)
def func(column):
    # column: pd.Series - 개별 컬럼의 값들을 Series로 받는다.
    ## 각 컬럼의 대표값을 반환. -> 숫자형: 평균, 문자열: 최빈값
    if column.dtype == 'object':
        return column.mode()[0]
    else:
        return column.mean()

In [186]:
flight.apply(func)#   #, axis=1 : 행이 전달

MONTH           6.220646
DAY            15.702096
WEEKDAY              월요일
WEEKDAY2        3.926862
AIRLINE               DL
ORG_AIR              ATL
DEST_AIR             LAX
SCHED_DEP    1387.984836
DEP_DELAY      10.921192
AIR_TIME      115.928576
DIST          872.900072
SCHED_ARR    1549.399832
ARR_DELAY       5.812315
DIVERTED        0.002342
CANCELLED       0.015062
dtype: object

In [182]:
flight.head()

,MONTH,DAY,WEEKDAY,WEEKDAY2,AIRLINE,ORG_AIR,DEST_AIR,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,1,1,목요일,4,WN,LAX,SLC,1625,58.0,94.0,590,1905,65.0,0,0
1,1,1,목요일,4,UA,DEN,IAD,823,7.0,154.0,1452,1333,-13.0,0,0
2,1,1,목요일,4,MQ,DFW,VPS,1305,36.0,85.0,641,1453,35.0,0,0
3,1,1,목요일,4,AA,DFW,DCA,1555,7.0,126.0,1192,1935,-7.0,0,0
4,1,1,목요일,4,WN,LAX,MCI,1720,48.0,166.0,1363,2225,39.0,0,0


In [180]:
if flight['WEEKDAY'].dtype == 'object':
    print(flight['WEEKDAY'].mode()[0])

월요일


In [191]:
flight.select_dtypes(include='number').apply(lambda x : x * 5)

,MONTH,DAY,WEEKDAY2,SCHED_DEP,DEP_DELAY,AIR_TIME,DIST,SCHED_ARR,ARR_DELAY,DIVERTED,CANCELLED
0,5,5,20,8125,290.0,470.0,2950,9525,325.0,0,0
1,5,5,20,4115,35.0,770.0,7260,6665,-65.0,0,0
2,5,5,20,6525,180.0,425.0,3205,7265,175.0,0,0
3,5,5,20,7775,35.0,630.0,5960,9675,-35.0,0,0
4,5,5,20,8600,240.0,830.0,6815,11125,195.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
58487,60,155,20,2575,25.0,830.0,7320,5225,-95.0,0,0
58488,60,155,20,9550,65.0,355.0,2070,10250,20.0,0,0
58489,60,155,20,9230,-30.0,230.0,1310,9780,-25.0,0,0
58490,60,155,20,2625,195.0,620.0,4535,4275,170.0,0,0


# cut()/qcut() - 연속형(실수)을 범주형으로 변환
- cut() : 지정한 값을 기준으로 구간을 나눠 그룹으로 묶는다. (0부터 10까지, 10부터 20까지, 20부터 30까지 등등...)
    - `pd.cut(x, bins,right=True, labels=None)`
    - 매개변수
        - **x**: 범주형으로 바꿀 대상. 1차원 배열형태(Series, 리스트, ndarray)의 자료구조
        - **bins**: 범주로 나눌때의 기준값(구간경계)들을 리스트로 묶어서 전달한다. (몇등분할꺼여?)
        - **right**: 구간경계의 오른쪽(True-기본)을 포함할지 왼쪽(False)을 포함할지
        - **labels**: 각 구간(범주)의 label을 리스트로 전달
            - 생략하면 범위를 범주명으로 사용한다. (ex: (10, 20], ()-포함안함, []-포함)
- qcut() :  대상배열의 최대값 ~ 최소값을 지정한 개수의 동등한 size(**원소의개수**)가 되도록 나눈다.
    - `pd.qcut(x, q, labels)`
    - 매개변수
        - **x**: 나눌 대상. 1차원 배열형태의 자료구조
        - **q**: 나눌 개수
        - **labels**: 각 구간(범주)의 label을 리스트로 전달

In [49]:
import numpy as np
np.random.seed(0)
age = np.random.randint(1, 100, 30)  # 1 ~ 100 사이 정수를 랜덤하게 30개 생성
tall = np.round(np.random.normal(170, 10, 30), 2)
# 평균: 170, 표준편차: 10 인 정규분포를 따르는 실수를 랜덤하게 30 생성
# 생성되는 값의 95는 150 ~ 190 사이의 실수

In [50]:
df = pd.DataFrame({
    "나이": age,
    "키":tall
})

In [52]:
#데이터프레임을 한번 출력해보자!
df

,나이,키
0,45,168.18
1,48,184.10
2,65,166.26
3,68,172.75
4,68,160.39
5,10,173.77
6,84,170.33
7,22,176.81
8,37,154.37
9,88,164.33


In [51]:
df.나이.value_counts()

89    3
68    2
10    2
88    2
45    1
47    1
70    1
81    1
21    1
73    1
78    1
26    1
38    1
82    1
59    1
40    1
66    1
48    1
13    1
71    1
37    1
22    1
84    1
65    1
80    1
Name: 나이, dtype: int64

In [212]:
df.키.value_counts()

키
168.18    1
184.10    1
179.96    1
162.24    1
166.14    1
176.40    1
159.14    1
165.37    1
171.75    1
164.30    1
187.87    1
183.95    1
159.22    1
171.49    1
175.66    1
185.35    1
184.63    1
170.47    1
166.67    1
185.14    1
167.58    1
164.33    1
154.37    1
176.81    1
170.33    1
173.77    1
160.39    1
172.75    1
166.26    1
150.67    1
Name: count, dtype: int64

In [213]:
df.키.mean()

170.843

In [217]:
df.나이.agg(['min', 'max'])

min    10
max    89
Name: 나이, dtype: int32

In [ ]:
df.나이

In [53]:
# cut
result = pd.cut(df.나이, bins=3) # bins: 몇등분 할지,나이를 기준으로 3등분한다는 소리이다.
# 지정한대로 그룹을 만든 다음에 개별값들을 그룹값으로 모두 처리한 결과를 반환.
result.value_counts()

(62.667, 89.0]      17
(36.333, 62.667]     7
(9.921, 36.333]      6
Name: 나이, dtype: int64

In [55]:
#그냥 results도 출력해 보면 재미 있을 것이다. 각 데이터가 어느 범주에 위치해 있는지 알려주는 데이터이다.

result

0     (36.333, 62.667]
1     (36.333, 62.667]
2       (62.667, 89.0]
3       (62.667, 89.0]
4       (62.667, 89.0]
5      (9.921, 36.333]
6       (62.667, 89.0]
7      (9.921, 36.333]
8     (36.333, 62.667]
9       (62.667, 89.0]
10      (62.667, 89.0]
11      (62.667, 89.0]
12      (62.667, 89.0]
13     (9.921, 36.333]
14    (36.333, 62.667]
15      (62.667, 89.0]
16    (36.333, 62.667]
17      (62.667, 89.0]
18    (36.333, 62.667]
19      (62.667, 89.0]
20      (62.667, 89.0]
21    (36.333, 62.667]
22     (9.921, 36.333]
23      (62.667, 89.0]
24      (62.667, 89.0]
25     (9.921, 36.333]
26     (9.921, 36.333]
27      (62.667, 89.0]
28      (62.667, 89.0]
29      (62.667, 89.0]
Name: 나이, dtype: category
Categories (3, interval[float64, right]): [(9.921, 36.333] < (36.333, 62.667] < (62.667, 89.0]]

In [ ]:
# 범위 표시 할때 
# (시작, 끝): 불포함 ==> opened
# [시작, 끝]: 포함   ==> closed

# (62.667, 89.0]  :  66.667 < 범위 <= 89.0

In [220]:
result[:5]

0    (36.333, 62.667]
1    (36.333, 62.667]
2      (62.667, 89.0]
3      (62.667, 89.0]
4      (62.667, 89.0]
Name: 나이, dtype: category
Categories (3, interval[float64, right]): [(9.921, 36.333] < (36.333, 62.667] < (62.667, 89.0]]

In [221]:
df.나이.iloc[:5]

0    45
1    48
2    65
3    68
4    68
Name: 나이, dtype: int32

In [58]:
result2 = pd.cut(df.나이, bins=3, right=False) # right=False 분위 값을 왼쪽(시작쪽)에 포함. 중요할 때도 있고 덜 중요할 때도 있고...
result2.value_counts()

[62.667, 89.079)    17
[36.333, 62.667)     7
[10.0, 36.333)       6
Name: 나이, dtype: int64

In [227]:
result3 = pd.cut(df.나이, bins=3, right=False, 
                labels=['나이대1', '나이대2', '나이대3']) # 나뉜 그룹에 이름들을 지정
result3.value_counts()

나이
나이대3    17
나이대2     7
나이대1     6
Name: count, dtype: int64

In [60]:
df2 = df.copy()

In [65]:
#새로운 컬럼을 만든다.
#label의 의미를 잘 생각해 보자.
df2['나이대'] = pd.cut(df['나이'],bins=3,labels=['나이대1','나이대2','나이대3'])
df2.groupby('나이대')['키'].mean() #나이대 별로 키의 평균을 구하자.

나이대
나이대1    172.556667
나이대2    170.065714
나이대3    170.558235
Name: 키, dtype: float64

In [68]:
df2['나이대'].value_counts().to_frame() #value_counts를 하는 것에 그치지 않고, to_frame을 써서 데이터를 더 쉽게 보이게 만들자.
#이를 간단히 말하면 도수분포표라고 한다.

,나이대
나이대3,17
나이대2,7
나이대1,6


In [70]:
#원하는 범위로 그룹을 나누기.

l =[0,20,40,60,100]
#0~20,20~40,40~60,60~100 등 이렇게 데이터 그룹을 나눈다.

result4 = pd.cut(df2['나이'],bins=l)


result4#이렇게 결과가 만들어졌음을 잘 보자.

0      (40, 60]
1      (40, 60]
2     (60, 100]
3     (60, 100]
4     (60, 100]
5       (0, 20]
6     (60, 100]
7      (20, 40]
8      (20, 40]
9     (60, 100]
10    (60, 100]
11    (60, 100]
12    (60, 100]
13      (0, 20]
14     (40, 60]
15    (60, 100]
16     (20, 40]
17    (60, 100]
18     (40, 60]
19    (60, 100]
20    (60, 100]
21     (20, 40]
22     (20, 40]
23    (60, 100]
24    (60, 100]
25      (0, 20]
26     (20, 40]
27    (60, 100]
28    (60, 100]
29    (60, 100]
Name: 나이, dtype: category
Categories (4, interval[int64, right]): [(0, 20] < (20, 40] < (40, 60] < (60, 100]]

In [73]:
result4.value_counts().sort_index()

(0, 20]       3
(20, 40]      6
(40, 60]      4
(60, 100]    17
Name: 나이, dtype: int64

In [75]:
#원하는 범위로 그룹을 나누기.

l =[0,20,40,60,100]
#0~20,20~40,40~60,60~100 등 이렇게 데이터 그룹을 나눈다.

#labels를 통해 데이터의 그룹을 나눈다.
result5 = pd.cut(df2['나이'],bins=l,labels=['청소년','청년','장년','노년'])


#result5의 데이터 분류 기준에 따라 데이터가 나뉘어 진다.
result5.value_counts().sort_index()

청소년     3
청년      6
장년      4
노년     17
Name: 나이, dtype: int64

In [76]:
#데이터를 새롭게 갱신하자. df2에 데이터를 새롭게 갱신하자.

df2['나이대2'] = result5
df2

,나이,키,나이대,나이대2
0,45,168.18,나이대2,장년
1,48,184.10,나이대2,장년
2,65,166.26,나이대3,노년
3,68,172.75,나이대3,노년
4,68,160.39,나이대3,노년
5,10,173.77,나이대1,청소년
6,84,170.33,나이대3,노년
7,22,176.81,나이대1,청년
8,37,154.37,나이대2,청년
9,88,164.33,나이대3,노년


In [79]:
df2.groupby('나이대2')['키'].agg(['min','max'])
#나이대별 키의 min,max값을 잘 구할 수 있다.

,min,max
나이대2,,
청소년,170.47,176.40
청년,154.37,176.81
장년,159.22,184.63
노년,150.67,187.87


In [82]:
#qcut 등분은 등분인데, 동일한 '원소의 개수'가 되도록 그룹을 나눈다.
#q는 quantity의 약자이다.
result = pd.qcut(df['나이'],q=4) #q가 몇 개로 나뉠지 지정한다.
result
result.value_counts()

(9.999, 38.5]    8
(80.75, 89.0]    8
(38.5, 67.0]     7
(67.0, 80.75]    7
Name: 나이, dtype: int64

In [84]:
result = pd.qcut(df['나이'],q=4,labels=['A','B','C','D'])
result.value_counts()

A    8
D    8
B    7
C    7
Name: 나이, dtype: int64

<b style='font-size:2em'>TODO </b>

In [ ]:
# 1: data/diamonds.csv 를 읽어 DataFrame으로 만든다.

# 2: price 컬럼을 '고가', '중가', '저가' 세개의 범주값을 가지는 "price_cate" 컬럼을 생성한다.

# 3 가격대(price_cate) 별 carat의 평균을 조회

# 4 가격대(price_cate)와 cut별 평균 가격(price)를 피봇테이블로 조회

# 5 cut, color, price_cate 별 carat의 평균을 피봇테이블로 조회